# SAC Ion Exchange Breakthrough Analysis

This notebook performs a complete SAC ion exchange cycle simulation and generates visualizations.

In [ ]:
# Parameters cell - will be replaced by papermill
water_analysis = {
    "flow_m3_hr": 100.0,
    "temperature_celsius": 25.0,
    "pH": 7.5,
    "pressure_bar": 1.0,
    "ca_mg_l": 80.0,
    "mg_mg_l": 25.0,
    "na_mg_l": 800.0,
    "cl_mg_l": 1400.0,
    "hco3_mg_l": 120.0
}

vessel_configuration = {
    "bed_volume_L": 1000.0,
    "service_flow_rate_bv_hr": 16.0,
    "vessel_id": "SAC-001",
    "number_service": 1,
    "number_standby": 1,
    "diameter_m": 1.5,
    "bed_depth_m": 2.0,
    "resin_volume_m3": 1.0,
    "freeboard_m": 0.5,
    "vessel_height_m": 2.5
}

target_hardness_mg_l_caco3 = 5.0

regeneration_config = {
    "enabled": True,
    "regenerant_type": "NaCl",
    "concentration_percent": 11.0,
    "flow_rate_bv_hr": 2.5,
    "mode": "staged_optimize",
    "regeneration_stages": 5,
    "target_recovery": 0.90
}

# Project root parameter (passed by papermill for path resolution)
project_root_str = None

In [ ]:
# Imports
import sys
import os
from pathlib import Path
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Add project root to path - robust approach
if 'IX_DESIGN_MCP_ROOT' in os.environ:
    project_root = Path(os.environ['IX_DESIGN_MCP_ROOT'])
elif 'project_root_str' in locals():
    # Parameter passed by papermill
    project_root = Path(project_root_str)
else:
    # Fallback: assume notebook is in notebooks/ directory
    try:
        project_root = Path(__file__).parent.parent
    except NameError:
        # __file__ not defined in Jupyter, use cwd fallback
        project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import simulation and plotting functions
from tools.sac_simulation import simulate_sac_phreeqc, SACSimulationInput, RegenerationConfig
from tools.sac_configuration import SACWaterComposition, SACVesselConfiguration
from tools.breakthrough_plotting import generate_cycle_plots

print("Imports completed successfully")
print(f"Project root: {project_root}")

In [ ]:
# Create input objects from parameters
water = SACWaterComposition(**water_analysis)
vessel = SACVesselConfiguration(**vessel_configuration)
regen = RegenerationConfig(**regeneration_config)

sim_input = SACSimulationInput(
    water_analysis=water,
    vessel_configuration=vessel,
    target_hardness_mg_l_caco3=target_hardness_mg_l_caco3,
    regeneration_config=regen
)

print("Input configuration created")
print(f"Flow rate: {water.flow_m3_hr} m³/hr")
print(f"Hardness: {water.ca_mg_l + water.mg_mg_l * 0.609} mg/L as CaCO₃")
print(f"Bed volume: {vessel.bed_volume_L} L")

In [ ]:
# Run simulation
print("\nRunning SAC simulation...")
results = simulate_sac_phreeqc(sim_input)

# Display key metrics
print("\n" + "="*50)
print("SIMULATION RESULTS")
print("="*50)
print(f"\nService Phase:")
print(f"  Breakthrough: {results.breakthrough_bv:.1f} BV")
print(f"  Service time: {results.service_time_hours:.1f} hours")
print(f"  Capacity factor: {results.phreeqc_determined_capacity_factor:.2f}")
print(f"  Utilization: {results.capacity_utilization_percent:.1f}%")

if results.regeneration_results:
    print(f"\nRegeneration Phase:")
    print(f"  Regenerant used: {results.regeneration_results.regenerant_consumed_kg:.1f} kg")
    print(f"  Final recovery: {results.regeneration_results.final_resin_recovery * 100:.1f}%")
    print(f"  Regeneration time: {results.regeneration_results.regeneration_time_hours:.1f} hours")
    print(f"\nTotal Cycle:")
    print(f"  Total time: {results.total_cycle_time_hours:.1f} hours")

In [ ]:
# Generate and display plots
print("\nGenerating breakthrough curves...")

try:
    plots = generate_cycle_plots(
        results.breakthrough_data,
        water,
        target_hardness_mg_l_caco3
    )
    
    # Display service breakthrough plot
    if 'service_breakthrough' in plots:
        display(plots['service_breakthrough'])
    
    # Display full cycle plot if available
    if 'full_cycle' in plots:
        display(plots['full_cycle'])
        
except Exception as e:
    print(f"Warning: Could not generate plots: {e}")
    print("Continuing with analysis...")

In [ ]:
# Create summary DataFrame for easy viewing
summary_data = {
    'Metric': [
        'Breakthrough (BV)',
        'Service Time (hours)',
        'Capacity Factor',
        'Regenerant Used (kg)',
        'Recovery (%)',
        'Total Cycle Time (hours)'
    ],
    'Value': [
        f"{results.breakthrough_bv:.1f}",
        f"{results.service_time_hours:.1f}",
        f"{results.phreeqc_determined_capacity_factor:.2f}",
        f"{results.regeneration_results.regenerant_consumed_kg:.1f}" if results.regeneration_results else "N/A",
        f"{results.regeneration_results.final_resin_recovery * 100:.1f}" if results.regeneration_results else "N/A",
        f"{results.total_cycle_time_hours:.1f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
display(summary_df)

In [ ]:
# Store results for extraction by papermill
notebook_results = {
    'status': 'success',
    'breakthrough_bv': results.breakthrough_bv,
    'service_time_hours': results.service_time_hours,
    'total_cycle_time_hours': results.total_cycle_time_hours,
    'capacity_factor': results.phreeqc_determined_capacity_factor,
    'final_recovery': results.regeneration_results.final_resin_recovery if results.regeneration_results else None,
    'regenerant_kg': results.regeneration_results.regenerant_consumed_kg if results.regeneration_results else None
}

# Store as JSON for easy extraction
notebook_results_json = json.dumps(notebook_results)
print(f"\nResults stored for extraction: {notebook_results_json}")